In [7]:
words=open('dataset/names.txt').read().splitlines()

In [8]:
words[:10]

['Abhishek',
 'Aman',
 'Harsh',
 'Ayush',
 'Aditi',
 'Anjali',
 'Shubham',
 'Anushka',
 'Rohit',
 'Saurabh']

In [9]:
min(len(w) for w in words)

2

In [10]:
max(len(w) for w in words)

14

In [22]:
b={}
for w in words:
    chs=["<S>"]+list(w)+["<E>"]
    for c1,c2 in zip(chs,chs[1:]):
        bigram=(c1,c2)
        b[bigram]=b.get(bigram,0)+1

In [37]:
#hide
sorted(b.items(),key=lambda k: -k[1])[:5]

[(('a', '<E>'), 304),
 (('s', 'h'), 294),
 (('a', 'n'), 272),
 (('<S>', 'S'), 258),
 (('h', 'a'), 220)]